# Homework Assignment 7

### Topics: 

In [2]:
import boto3
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import precision_recall_cutoff as prc
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

In [3]:
## 1. Using pandas to read the training and testing data files

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

file_key = 'train.csv'
file_key2 = 'test.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')

train = pd.read_csv(file_content_stream, sep = '|')
test = pd.read_csv(file_content_stream2, sep = '|')

train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [4]:
## Creating variables from previous homework assignments

## Training set:

## Variable 1 (from decision tree)
train['Interaction_1'] = np.where((train['trustLevel'] <= 1.5) & (train['scannedLineItemsPerSecond'] <= 0.012) & 
                                  (train['lineItemVoids'] <= 10.5), 0, 1)
## Variable 2 (from decision tree)
train['Interaction_2'] = np.where((train['trustLevel'] <= 1.5) & (train['scannedLineItemsPerSecond'] > 0.012) & 
                                  (train['totalScanTimeInSeconds'] <= 895.0), 0, 1)
## Variable 3 (from decision tree)
train['Interaction_3'] = np.where((train['trustLevel'] > 1.5) & (train['grandTotal'] <= 99.145) & 
                                  train['trustLevel'] <= 2.5, 1, 0)
## Variable 4 (from decision tree)
train['Interaction_4'] = np.where((train['trustLevel'] > 1.5) & (train['grandTotal'] > 99.145) & 
                                  train['valuePerSecond'] <= 0.06, 1, 0)
## Variable 5 - Low trustLevel (all frauds came from trustLevel = 1 or 2)
train['lowTrust'] = np.where(train['trustLevel'] <= 2, 1, 0)

## Variable 6 - Made a quantity modification
train['madeModification'] = np.where(train['quantityModifications'] > 0, 1, 0)

## Variable 7 - Attempted a scan without registration
train['madeScansWithoutRegistration'] = np.where(train['scansWithoutRegistration'] > 0, 1, 0)

## Variable 8 - High or low totalScanTimeInSeconds (huge differnece in mean and median values for fraud/not fraud in this field)
train['lowTotalScanTime'] = np.where(train['totalScanTimeInSeconds'] < 1000, 1, 0)

## Varibales from strong heredity principle
train['Heredity_Feature_1'] = train['trustLevel'] * train['lowTrust']
train['Heredity_Feature_2'] = train['trustLevel'] * train['scannedLineItemsPerSecond']
train['Heredity_Feature_3'] = train['lowTrust'] * train['scannedLineItemsPerSecond']

## Heredity_Feature_3: all observations less than 0.012 are not fraud in this tree
train['New_Interaction_1'] = np.where(train['Heredity_Feature_3'] <= 0.012, 1, 0)

## Lots of positove observations when Heredity_Feature_3 > 0.012, totalScanTimeInSeconds > 1298.0, and trustLevel <= 1.5
train['New_Interaction_2'] = np.where((train['Heredity_Feature_3'] > 0.012) & (train['totalScanTimeInSeconds'] > 1298.0) 
                                      & (train['trustLevel'] < 1.5), 1, 0)

## Mostly all negative observations when Heredity_Feature_3 > 0.012, totalScanTimeInSeconds <= 1298.0, and Heredity_Feature_1 > 0.119
train['New_Interaction_3'] = np.where((train['Heredity_Feature_3'] > 0.012) & (train['totalScanTimeInSeconds'] <= 1298.0) 
                                      & (train['Heredity_Feature_1'] > 1.5), 1, 0)

In [5]:
## Testing set:

## Variable 1
test['Interaction_1'] = np.where((test['trustLevel'] <= 1.5) & (test['scannedLineItemsPerSecond'] <= 0.012) & 
                                  (test['lineItemVoids'] <= 10.5), 0, 1)
## Variable 2
test['Interaction_2'] = np.where((test['trustLevel'] <= 1.5) & (test['scannedLineItemsPerSecond'] > 0.012) & 
                                  (test['totalScanTimeInSeconds'] <= 895.0), 0, 1)
## Variable 3
test['Interaction_3'] = np.where((test['trustLevel'] > 1.5) & (test['grandTotal'] <= 99.145) & 
                                  test['trustLevel'] <= 2.5, 1, 0)
## Variable 4
test['Interaction_4'] = np.where((test['trustLevel'] > 1.5) & (test['grandTotal'] > 99.145) & 
                                  test['valuePerSecond'] <= 0.06, 1, 0)
## Variable 5 - Low trustLevel
test['lowTrust'] = np.where(test['trustLevel'] <= 2, 1, 0)

## Variable 6 - Made a quantity modification
test['madeModification'] = np.where(test['quantityModifications'] > 0, 1, 0)

## Variable 7 - Attempted a scan without registration
test['madeScansWithoutRegistration'] = np.where(test['scansWithoutRegistration'] > 0, 1, 0)

## Variable 8 - High or low totalScanTimeInSeconds
test['lowTotalScanTime'] = np.where(test['totalScanTimeInSeconds'] < 1000, 1, 0)

## Varibales from strong heredity principle
test['Heredity_Feature_1'] = test['trustLevel'] * test['lowTrust']
test['Heredity_Feature_2'] = test['trustLevel'] * test['scannedLineItemsPerSecond']
test['Heredity_Feature_3'] = test['lowTrust'] * test['scannedLineItemsPerSecond']

test['New_Interaction_1'] = np.where(test['Heredity_Feature_3'] <= 0.012, 1, 0)

test['New_Interaction_2'] = np.where((test['Heredity_Feature_3'] > 0.012) & (test['totalScanTimeInSeconds'] > 1298.0) 
                                      & (test['trustLevel'] < 1.5), 1, 0)

test['New_Interaction_3'] = np.where((test['Heredity_Feature_3'] > 0.012) & (test['totalScanTimeInSeconds'] <= 1298.0) 
                                      & (test['Heredity_Feature_1'] > 1.5), 1, 0)

In [ ]:
## 2.  Using the train data-frame (including the top 7 features from homework assignment 6) to do the following:

## (i) Considering a model to predict fraud:
## With the top 5 important features and using the GridSearchCV function with cv = 3 to run a hyper-parameter tuning procedure on the model
## With the top 6 important features and using the GridSearchCV function with cv = 3 to run a hyper-parameter tuning procedure on the model
## With the top 7 important features and using the GridSearchCV function with cv = 3 to run a hyper-parameter tuning procedure on the model

## Evaluating the models on the associated cost function from the Data Mining Cup task site








From above three scenarios, identify the best model; that is, the model model (input features
and hyper-parameters) that has the best performance.